In [55]:
import os
import glob
from datetime import date

from acrg.name import name
from acrg.config.paths import Paths

## Set up information

Information to be used throughout the notebook

Reference values are taken from Rodenbeck et al., 2008:

$X_0^{CO_2} = 350$ \
$X_0^{N_2} = 790190$ \
$X_0^{O_2} = 209460$

In [56]:
year = 2020

ref_values = {'co2': 350, 'n2': 790190, 'o2': 209460}
ref_values['o2_n2'] = ref_values['o2'] / ref_values['n2']

## Get the data

Get the netcdf files for the apo and co2 boundary conditions and open using xarray

In [57]:
bc_dir = {'apo': os.path.join(Paths.lpdm, 'bc', 'EUROPE')}
bc_dir['co2'] = os.path.join(bc_dir['apo'], 'co2_JenaCarboscope')

bc_files = {spec: glob.glob(os.path.join(directory, f'{spec}_EUROPE_{year}01.nc'))[0]
            for spec, directory in bc_dir.items()}

[print(f'{spec}: {bc}') for spec, bc in bc_files.items()]

bc_data = {spec: name.open_ds(bc) for spec, bc in bc_files.items()}

apo: /group/chemistry/acrg/LPDM/bc/EUROPE/apo_EUROPE_202001.nc
co2: /group/chemistry/acrg/LPDM/bc/EUROPE/co2_JenaCarboscope/co2_EUROPE_202001.nc


In [58]:
bc_data['apo'] = bc_data['apo'] / 0.20946 if (abs(bc_data['apo'].vmr_n)<100).all() else bc_data['apo']
bc_data['apo']

<xarray.Dataset>
Dimensions:  (time: 1464, height: 20, lon: 391, lat: 293)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 ... 2020-12-31T18:00:00
  * height   (height) float32 500.0 1.5e+03 2.5e+03 ... 1.85e+04 1.95e+04
  * lon      (lon) float32 -97.9 -97.55 -97.2 -96.84 ... 38.32 38.68 39.03 39.38
  * lat      (lat) float32 10.73 10.96 11.2 11.43 ... 78.36 78.59 78.82 79.06
Data variables:
    vmr_n    (time, height, lon) float64 -382.3 -382.1 -382.0 ... -370.7 -370.8
    vmr_s    (time, height, lon) float64 -394.1 -394.7 -395.4 ... -372.8 -372.7
    vmr_e    (time, height, lat) float64 -359.2 -359.2 -359.2 ... -369.8 -369.8
    vmr_w    (time, height, lat) float64 -389.8 -389.6 -389.5 ... -370.1 -370.1
Attributes:
    title:         JenaCarboscope APO volume mixing ratios at domain edges
    author:        vf20487
    date_created:  2022-07-13
    history:       Inversion code by Christian Roedenbeck
    references:    http://www.bgc-jena.mpg.de/CarboScope/

## Calculate the O2 boundary conditions

Use the APO equation to calculate the $\delta (O2/N2)$ boundary conditions:

$\delta (APO) = \delta (O2/N2) + 1.1 / X_O^{O2} (CO_2 - X_0^{CO_2}) \times 10^6$

\
Then use the $\delta (O2/N2)$ equation to calculate the O2 boundary conditions:

$\delta (O2/N2) = \frac{(O_2 / N_2)_{sample} - (O_2 / N_2)_{refence}}{(O_2 / N_2)_{refence}}$

In [59]:
per_meg_conversion = 1e6/ref_values['o2']
print(f'per meg conversion: {per_meg_conversion}')
del_o2_n2 = bc_data['apo'] * per_meg_conversion - 1.1 * per_meg_conversion * (bc_data['co2'] - ref_values['co2'])
bc_o2 = (del_o2_n2 * ref_values['o2_n2'] * 1e-6 + ref_values['o2_n2']) * ref_values['n2']

per meg conversion: 4.774181227919412


## Save to netcdf

Add attributes to the O2 dataset including the sim from which the bcs are estimated and the date created.

Create a standard filename in the same format as those for other species in /user/work/chxmr/shared/LPDM/bc/EUROPE .\
Save the dataset to a netcdf file in the shared directory.

In [60]:
# copy the CO2 attributes
o2_attrs = bc_data['co2'].attrs
# edit the species in the attributes title
o2_attrs['title'] = 'O2'.join(bc_data['co2'].attrs['title'].split('CO2'))
# add a description to explain how the bcs were estimated
o2_attrs['description'] = 'Estimated from Jena Carboscope CO2 and APO boundary conditions using APO equation'
# edit the creation date
o2_attrs['date_created'] = str(date.today())

# add the attributes to the O2 dataset
bc_o2 = bc_o2.assign_attrs(o2_attrs)

# create a standardised filename
filename = os.path.join(bc_dir['apo'], f'o2_EUROPE_{year}01.nc')

if os.path.isfile(filename):
    print(f'Boundary condition file {filename} already exists.')
    answer = input("You are about to overwrite an existing file, do you want to continue? Y/N ")
    save = False if answer.upper() == 'N' else True
else:
    save = True

# save the file
if save:
    print(f'Saving to: {filename}')
    bc_o2.to_netcdf(filename)
else:
    print('Not overwriting file')

Saving to: /group/chemistry/acrg/LPDM/bc/EUROPE/o2_EUROPE_202001.nc
